In [3]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict , Annotated
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import operator

In [4]:
load_dotenv()  # take environment variables from .env.

True

In [6]:
model = ChatOpenAI(model="gpt-4o-mini")

In [7]:
class EvaluationModel(BaseModel):
    feedback: str = Field(description="Feedback on the essay quality")
    score: int = Field(ge=1, le=10, description="Score between 1 and 10")

In [8]:
structured_model = model.with_structured_output(EvaluationModel)

In [ ]:
class EassayGen(TypedDict):
    topic: str
    essay_generation: str
    language_feedback: str
    analysis_feedback: str
    clearity_feedback: str
    summary: str
    scores: Annotated[list[int],operator.add]
    avg_score: float

In [19]:
def generate_essay(state: EassayGen) -> EassayGen:
    prompt = f'Write a detailed essay on the topic: {state["topic"]}'
    response = model.invoke(prompt)
    return {'essay_generation': response}

In [22]:
def evaluate_language(state: EassayGen):
    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay_generation"]}'
    response= structured_model.invoke(prompt)
    return {'analysis_feedback': response.feedback, 'scores': [response.score]}

In [ ]:

def evaluate_thought(state: EassayGen):

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay_generation"]}'
    response = structured_model.invoke(prompt)

    return {'clarity_feedback': response.feedback, 'individual_scores': [response.score]}

In [ ]:
def evaluate_analysis(state: EassayGen):
    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay_generation"]}'
    response= structured_model.invoke(prompt)
    return {'language_feedback': response.feedback, 'scores': [response.score]}

In [ ]:
graph = StateGraph(EassayGen)
graph.add_node('essay_generation', generate_essay)
graph.add_node('essay_generation', generate_essay)
graph.add_node('essay_generation', generate_essay)

graph.add_edge(START, 'essay_generation')

<class '__main__.EassayGen'>
